In [2]:

from datetime import datetime, timedelta
import requests
import pandas as pd
import json


start_steam_id = 76561197960435537
number_of_ids = 10
api_key = "22AB7B290037490813DF382FD4F94FA4"

# Container for all user summary URLs
users = []

for i in range(number_of_ids):
        # Generate the URL for each Steam ID
        steam_id = str(start_steam_id + i)
        url = f'https://api.steampowered.com/ISteamUser/GetPlayerSummaries/v0002/?key={api_key}&steamids={steam_id}'

        # Fetch data from the URL
        response = requests.get(url)
        if response.status_code == 200:
            # Parse the JSON response and append the player data to the users list
            player_data = response.json().get("response", {}).get("players", [])
            for player in player_data:
                users.append(player)
        else:
            print(f"Failed to fetch data for Steam ID: {steam_id}")
# Convert the users list to a DataFrame
df = pd.DataFrame(users)

# Print the DataFrame to verify the data
df.dtypes

steamid                      object
communityvisibilitystate      int64
personaname                  object
profileurl                   object
avatar                       object
avatarmedium                 object
avatarfull                   object
avatarhash                   object
personastate                  int64
primaryclanid                object
timecreated                 float64
personastateflags           float64
profilestate                float64
realname                     object
loccountrycode               object
locstatecode                 object
dtype: object

-Connection with tables
-Watch and learn and reccomendation model form book and youtube - making test prediction with random data


-trying to create proeduction model 

-finalyzin the project 



-being the next rpoject but with snowflake and azure env 


In [5]:
import pandas as pd
import json
import pyodbc
from pyspark.sql import SparkSession
import pyodbc
import sqlalchemy
from sqlalchemy import create_engine
import urllib
from sqlalchemy.exc import OperationalError


In [6]:
# Create a Spark session
spark_games = SparkSession.builder \
    .appName("RecomendationForPlayers") \
    .getOrCreate()

In [7]:
server = r"DESKTOP-06UQ0CT\DATA_CENTER"  # Adjust server name
username = "MichalM"
password = "!Everest2021@!"
database = "SteamDB"
driver = "ODBC+Driver+18+for+SQL+Server"



def serverConnection(server,username,password,database,driver, table):
    # Build the connection string for SQLAlchemy
    connection_string = f"mssql://{username}:{urllib.parse.quote_plus(password)}@{server}/{database}?driver={driver}&Encrypt=no&TrustServerCertificate=yes"

    engine = create_engine(connection_string)
    con = engine.connect()
    
    with engine.connect() as connection:
        # Execute the SQL query and return the result as a DataFrame
        query = f"SELECT * FROM [SteamDB].[dbo].[{table}];"
        result_df = pd.read_sql(query, connection)
    
    return result_df



SteamGamesDescriptionDF = serverConnection(server,username,password,database,driver,'SteamGamesDescriptionFix')

SteamUsersDF = serverConnection(server,username,password,database,driver,'SteamUsers')

SteamUsersGamesDF = serverConnection(server,username,password,database,driver,'SteamUsersGames') 

SteamGamesDF = serverConnection(server,username,password,database,driver,'SteamGames')
    

In [4]:
SteamGamesDescriptionsparkDF = spark_games.createDataFrame(SteamGamesDescriptionDF)

SteamGamesDescriptionsparkDF.printSchema()



root
 |-- name: string (nullable = true)
 |-- steam_appid: long (nullable = true)
 |-- required_age: string (nullable = true)
 |-- is_free: boolean (nullable = true)
 |-- short_description: string (nullable = true)
 |-- supported_languages: string (nullable = true)
 |-- header_image: string (nullable = true)
 |-- capsule_image: string (nullable = true)
 |-- pc_requirements: string (nullable = true)
 |-- developers: string (nullable = true)
 |-- publishers: string (nullable = true)
 |-- metacritic: string (nullable = true)
 |-- currency: string (nullable = true)
 |-- initial_price: double (nullable = true)
 |-- final_price: double (nullable = true)
 |-- discount_percent: double (nullable = true)
 |-- initial_formatted: string (nullable = true)
 |-- final_formatted: string (nullable = true)
 |-- windows: string (nullable = true)
 |-- mac: string (nullable = true)
 |-- linux: string (nullable = true)



In [33]:
SteamGamesDescriptionsparkDF.show()

+--------------------+-----------+------------+-------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+--------+-------------+-----------+----------------+-----------------+---------------+-------+-----+-----+
|                name|steam_appid|required_age|is_free|   short_description| supported_languages|        header_image|       capsule_image|     pc_requirements|          developers|          publishers|metacritic|currency|initial_price|final_price|discount_percent|initial_formatted|final_formatted|windows|  mac|linux|
+--------------------+-----------+------------+-------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+--------+-------------+-----------+----------------+-----------------+---------------+-------+-----+-----+
|The Monstrous Hor...|    2099790|      